In [6]:
import syft as sy
import tenseal as ts
import pytest
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient

sy.load_lib("tenseal")

In [8]:
client = connect(
    url="http://localhost:5000", # Domain Address
    credentials={"email":"admin@email.com", "password":"pwd123"},
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
    client_type=DomainClient) # Domain Client type

In [2]:
context = ts.Context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2**40
context

In [3]:
v1 = [0, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 0]

enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)
(enc_v1, enc_v2)

(<tenseal.tensors.ckksvector.CKKSVector at 0x7f623b413730>,
 <tenseal.tensors.ckksvector.CKKSVector at 0x7f623b409a30>)

In [9]:
# tag them so our partner can easily reference it
ctx_ptr = context.send(client, searchable=True, tags=["context"])
enc_v1_ptr = enc_v1.send(client, searchable=True, tags=["enc_v1"])
enc_v2_ptr = enc_v2.send(client, searchable=True, tags=["enc_v2"])

In [10]:
client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 6d9c4fa096e74d70831e62b4a6a122cf>,[context],,<class 'tenseal.enc_context.Context'>
1,<UID: 4b3c9eb00bf046d4bba22ea7a9cc8cc5>,[enc_v1],,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,<UID: a4cd46b267114065b839ba7ea1af97e4>,[enc_v2],,<class 'tenseal.tensors.ckksvector.CKKSVector'>


###                                                           Checkpoint 1

In [11]:
client.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[context],I would like to get the context,<UID: 2e0039690ed74005b5096d96f9e64564>,<UID: 6d9c4fa096e74d70831e62b4a6a122cf>,<class 'tenseal.enc_context.Context'>
1,[enc_v1],I would like to get first vector,<UID: 38b1579ac12a421b955cffb757151920>,<UID: 4b3c9eb00bf046d4bba22ea7a9cc8cc5>,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,[enc_v2],I would like to get second vector,<UID: e1d4ef67223046cda0e941f237e253df>,<UID: a4cd46b267114065b839ba7ea1af97e4>,<class 'tenseal.tensors.ckksvector.CKKSVector'>


In [12]:
client.requests[0].accept()
client.requests[0].accept()
client.requests[0].accept()

In [14]:
client.requests.pandas

""


### Checkpoint 2

In [15]:
# Validate the encrypted add
result_add = client.store["result_add"].get(delete_obj=False)
result_add.link_context(context)

result_add

In [16]:
decrypted_result = result_add.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 + v2 for v1, v2 in zip(v1, v2)]

decrypted_result

[3.9999999995135243,
 4.000000000248688,
 3.999999998543466,
 3.9999999993537814,
 3.9999999999321147]

In [18]:
result_iadd = client.store["result_iadd"].get(delete_obj=False)
result_iadd.link_context(context)

decrypted_result = result_iadd.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 + v2 for v1, v2 in zip(v1, [10, 10, 10, 10, 10])]

decrypted_result

[10.000000000359641,
 11.000000000106214,
 11.999999999385691,
 13.000000000389708,
 14.000000000032443]

In [19]:
# Validate the encrypted subtraction

result_sub = client.store["result_sub"].get(delete_obj=False)
result_sub.link_context(context)

decrypted_result = result_sub.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 - v2 for v1, v2 in zip(v1, v2)]

decrypted_result

[-3.9999999987795594,
 -2.0000000000272924,
 1.3477974292186445e-10,
 2.0000000014695583,
 4.000000000041847]

In [20]:
# Validate the encrypted multiplication

result_mul = client.store["result_mul"].get(delete_obj=False)
result_mul.link_context(context)

decrypted_result = result_mul.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 * v2 for v1, v2 in zip(v1, v2)]

decrypted_result



[1.3283536492991743e-09,
 3.000000403307595,
 4.000000532791146,
 3.0000003984401156,
 -2.0960744251397045e-10]

In [21]:
# Validate the encrypted power

result_pow = client.store["result_pow"].get(delete_obj=False)
result_pow.link_context(context)

decrypted_result = result_pow.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v ** 3 for v in v1]

decrypted_result

[1.4695462624558786e-09,
 1.0000008073158178,
 8.000006430206803,
 27.00002173587953,
 64.00005150083823]

In [22]:
# Validate the encrypted negation

result_neg = client.store["result_neg"].get(delete_obj=False)
result_neg.link_context(context)

decrypted_result = result_neg.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [-v for v in v1]

decrypted_result



[-3.6698222238840117e-10,
 -1.0000000001106983,
 -1.9999999993391218,
 -3.0000000004116703,
 -3.999999999986981]

In [23]:
# Validate the encrypted polynomial evaluation for 1 + X^2 + X^3

result_poly = client.store["result_poly"].get(delete_obj=False)
result_poly.link_context(context)

decrypted_result = result_poly.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [1 + v**2 + v**3 for v in v1]

decrypted_result

[1.000000001823338,
 3.000000942843002,
 13.000006963971988,
 37.00002294609983,
 81.00005364751098]